## Apache kafka - wprowadzenie

Apache kafka to ,,serwer'' oraz ,,baza danych''.

Domyślnym adresem naszego serwera jest `broker:9092`

Nasza baza danych będzie posługiwała się tabelami, które w Apache Kafka nazywamy `topicami`.

Zarządzanie Kafką odbywa się z wykorzystaniem skryptów. W naszym przypadku będą to skrypty  `*.sh`.


### Sprawdź listę topiców

Pamiętaj aby przejść do katalogu domowego.

```bash
cd ~ 
kafka/bin/kafka-topics.sh --list --bootstrap-server broker:9092
```

### Utwórz nowy topic o nazwie `mytopic`

```bash
kafka/bin/kafka-topics.sh --create --topic mytopic  --bootstrap-server broker:9092
```

### Utwórz producenta 

Ten skrypt pozwoli Ci wprowadzać eventy ręcznie przez terminal.
Opcje `--property` są dodatkowe i wprowadzone tylko do analizy przeprowadzanej w tym przykładzie.

```bash
kafka/bin/kafka-console-producer.sh --bootstrap-server broker:9092 --topic mytopic --property "parse.key=true" --property "key.separator=:"
```

### Consument w Sparku

Otwórz nowy terminal w miejscu gdzie znajduje się plik test_key_value.py i uruchom program Consumenta na Sparku

Pamiętaj, iż Apache Spark nie posiada domyślnego konektora do Kafki dlatego uruchom proces poprzez komendę spark-submit i pobierz odpowiedni pakiet w scali.

```bash
spark-submit --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0  test_key_value.py
```

5. W terminalu z uruchominionym producentem wpisz teskt w postaci: 
```bash
jan:45
alicja:20
```
i sprawdz co pojawia się w oknie aplikacji consumenta.

Po zakończeniu pokazu ctrl+c zamyka zarówno okno producenta jak i aplikacje sparkową